Create a ResNet18 for our simulation of a malicious network attack

code referenced from: \
https://github.com/samcw/ResNet18-Pytorch/blob/master/ResNet18.ipynb \
https://github.com/kuangliu/pytorch-cifar/blob/master/main.py \
Assignment 2 in class

In [23]:
import torch
import torchvision

from torchsummary import summary

import numpy as np

a bit to make sure we are using the right python environment \
and if cuda is available

In [2]:
import sys
sys.version

'3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:49:16) [MSC v.1929 64 bit (AMD64)]'

In [3]:
torch.cuda.is_available()

True

In [4]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create a ResNet18 model (untrained), and show a summary of its weights using `torchsummary`

In [10]:
res18 = torchvision.models.resnet18().cuda() if torch.cuda.is_available() else torchvision.models.resnet18()

In [12]:
# check if the parameters are in cuda
next(res18.parameters()).is_cuda

True

In [11]:
summary(res18, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

Train it using CIFAR10 dataset, initially without a backdoor

In [15]:
import torchvision.transforms as transforms

# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
# )

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [16]:
cifar10_train = torchvision.datasets.CIFAR10('datasets/cifar_10', download=True, transform=transform_train)
# cifar10_test = torchvision.datasets.CIFAR10('datasets/cifar_10', train=False, download=True, transform=transform)

Files already downloaded and verified


In [17]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=4)

In [18]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [19]:
# copied from assignment 2
def compute_accuracy(prediction,gt_logits):
    pred_idx = np.argmax(prediction,1,keepdims=True)
    matches = pred_idx == gt_logits[:,None]
    acc = matches.mean()
    return acc

In [20]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(res18.parameters(),lr=5e-4)

In [25]:
for epoch in range(10):
    res18.train()
    it = 0
    # a very standard training loop
    for inputs, label in train_loader:
        # move data to cuda device
        inputs, label = inputs.to(device), label.to(device)
        
        optimizer.zero_grad()
        pred = res18(inputs)
        loss = loss_fn(pred,label)
        accuracy = compute_accuracy(pred.cpu().detach().numpy(),label.cpu().detach().numpy())
        # accuracy = compute_accuracy(pred,label)
        loss.backward()
        optimizer.step()

        print(f'Epoch: {epoch}, Iteration: {it} | Loss: {loss.item()} | Accuracy: {accuracy}')
        it += 1

Epoch: 0, Iteration: 0 | Loss: 7.0452117919921875 | Accuracy: 0.0
Epoch: 0, Iteration: 1 | Loss: 6.993470668792725 | Accuracy: 0.0
Epoch: 0, Iteration: 2 | Loss: 7.00122594833374 | Accuracy: 0.0
Epoch: 0, Iteration: 3 | Loss: 6.998387336730957 | Accuracy: 0.0
Epoch: 0, Iteration: 4 | Loss: 7.038463115692139 | Accuracy: 0.0
Epoch: 0, Iteration: 5 | Loss: 7.06461763381958 | Accuracy: 0.0
Epoch: 0, Iteration: 6 | Loss: 7.038190841674805 | Accuracy: 0.0
Epoch: 0, Iteration: 7 | Loss: 6.814189434051514 | Accuracy: 0.0
Epoch: 0, Iteration: 8 | Loss: 6.954325199127197 | Accuracy: 0.0
Epoch: 0, Iteration: 9 | Loss: 6.882272243499756 | Accuracy: 0.0
Epoch: 0, Iteration: 10 | Loss: 7.015179634094238 | Accuracy: 0.0
Epoch: 0, Iteration: 11 | Loss: 6.850745677947998 | Accuracy: 0.0
Epoch: 0, Iteration: 12 | Loss: 6.800398349761963 | Accuracy: 0.0
Epoch: 0, Iteration: 13 | Loss: 6.700173377990723 | Accuracy: 0.0
Epoch: 0, Iteration: 14 | Loss: 6.645021438598633 | Accuracy: 0.0
Epoch: 0, Iteration: 